In [2]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = '/content/drive/MyDrive/Colab Notebooks/Laptop_price.csv'  # Убедитесь, что путь к файлу верный
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

# Определите числовые и категориальные признаки
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

# Трансформеры для числовых и категориальных данных
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Комбинируем трансформеры
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

# Создаем пайплайн
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])

# Обучаем модель
pipeline.fit(X_train, y_train)

# Сохраняем модель
joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')

['/content/drive/MyDrive/laptop_price_model.pkl']

In [41]:
rm -Rfv .git

removed directory '.git/branches'
removed '.git/hooks/prepare-commit-msg.sample'
removed '.git/hooks/pre-applypatch.sample'
removed '.git/hooks/pre-commit.sample'
removed '.git/hooks/update.sample'
removed '.git/hooks/pre-merge-commit.sample'
removed '.git/hooks/pre-push.sample'
removed '.git/hooks/fsmonitor-watchman.sample'
removed '.git/hooks/post-update.sample'
removed '.git/hooks/commit-msg.sample'
removed '.git/hooks/pre-rebase.sample'
removed '.git/hooks/applypatch-msg.sample'
removed '.git/hooks/pre-receive.sample'
removed '.git/hooks/push-to-checkout.sample'
removed directory '.git/hooks'
removed '.git/info/exclude'
removed directory '.git/info'
removed '.git/logs/HEAD'
removed '.git/logs/refs/heads/main'
removed directory '.git/logs/refs/heads'
removed directory '.git/logs/refs'
removed directory '.git/logs'
removed '.git/index'
removed '.git/COMMIT_EDITMSG'
removed '.git/config'
removed '.git/objects/2b/8bbafe1a0bbb965185674b92ccc0e66c9e031e'
removed directory '.git/objects/2

In [42]:
!git branch -m main
!git config --global user.email "supertestuser1230@mail.ru"
!git config --global user.name "supertestuser1230"
!git init

fatal: not a git repository (or any of the parent directories): .git
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [43]:
!git add /content/drive/MyDrive/laptop_price_model.pkl
!git add /content/drive/MyDrive/Colab\ Notebooks/Laptop_price.csv
!git add /content/drive/MyDrive/Colab\ Notebooks/Pipeline.ipynb

In [44]:
!git commit -m "Добавлен ML-пайплайн"

[master (root-commit) a755eec] Добавлен ML-пайплайн
 3 files changed, 1002 insertions(+)
 create mode 100644 drive/MyDrive/Colab Notebooks/Laptop_price.csv
 create mode 100644 drive/MyDrive/Colab Notebooks/Pipeline.ipynb
 create mode 100644 drive/MyDrive/laptop_price_model.pkl


In [36]:
!git remote add origin https://github.com/supertestuser1230/pipeline-model.git

In [45]:
!git push -u origin master

fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [ ]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
import uvicorn
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "/content/drive/MyDrive/laptop_price_model.pkl"  # Убедитесь, что путь правильный
model = joblib.load(model_path)

@app.post("/predict")  # Добавлен пропущенный слэш
async def predict(file: UploadFile = File(...)):
    # Проверка типа файла
    if not file.filename.endswith('.csv'):
        return {"error": "Please upload a CSV file"}

    try:
        content = await file.read()
        df = pd.read_csv(BytesIO(content))

        # Проверка, что данные соответствуют ожиданиям модели
        if df.empty:
            return {"error": "The file is empty"}

        # Получение предсказаний
        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}  # Исправлена опечатка в ключе

    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)



Overwriting app.py


In [ ]:
cat app.py


from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
import uvicorn
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "/content/drive/MyDrive/laptop_price_model.pkl"  # Убедитесь, что путь правильный
model = joblib.load(model_path)

@app.post("/predict")  # Добавлен пропущенный слэш
async def predict(file: UploadFile = File(...)):
    # Проверка типа файла
    if not file.filename.endswith('.csv'):
        return {"error": "Please upload a CSV file"}
    
    try:
        content = await file.read()
        df = pd.read_csv(BytesIO(content))
        
        # Проверка, что данные соответствуют ожиданиям модели
        if df.empty:
            return {"error": "The file is empty"}
            
        # Получение предсказаний
        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}  # Исправлена опечатка в ключе
    
    except Exception as e:
        return {"error": f"An error occurred: {st

In [ ]:
!ngrok config add-authtoken <ngrok_auth_token>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

In [ ]:
!cat fastapi.log


INFO:     Started server process [28335]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     83.239.63.182:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     83.239.63.182:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     83.239.63.182:0 - "POST /predict HTTP/1.1" 200 OK


In [ ]:

from pyngrok import ngrok
# Подключаем публичный URL
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)


API доступно по адресу: NgrokTunnel: "https://69a4-34-173-139-8.ngrok-free.app" -> "http://localhost:8000"
